# Re-activation test model

## Overview:

You are being provided a file with combined series of historical mail promotional files.  The purpose of these campaigns are to “re-activate” people who used to donate a lot to a charity, however in recent months or years have stopped giving. Heuristic financial information from within many charities suggest that there is a lower cost to raise a dollar by re-activating lapsed donors then say acquiring a cold donor who is unfamiliar with the charity.  Many charities have large lapsed data segments in their databases making this a field of strong interest for the charity.

## Goal:

Generally your goal in a production scenario could be to use a model to drive the most names (largest lapsed mailing) possible at break even, since long term value suggests re-activated donors go on to become very valuable.
<br><br>
In the interest of brevity we will only be looking at Area under the ROC to determine success and keep the task simple in its goal.  Although, the actual response amount (GIFT) is provided in the data (except for blind validation file), this is going to be response driven only.
<br><br>
Although we are looking to see your model “work”, we are most interested in your process, questions you may have as you work, creativity and process.  So aside from delivering the blind validation file, please deliver any notes and data visualizations from the project so we can follow your thought process.  In other words sell us on your methodology. 
<br><br>
You may use any learning algorithm of your choosing and environment.  Although, this is a classic supervised learning scenario, we are not putting any constraints on how you choose to build a solution. 

## Data:

You have 2 files provided.  1 for training and validation and 1 for blind validation to send us back when you have provided a score.
<br><br>
Much of the work on the data has been done for you.  Using the client’s internal database we have created accurate snapshots for you of what the data looked like for each campaign at the time it was sent with of course who responded.   As a ‘data scientist’ with VeraData you will be provided with an expert to handle ETL and work with you so you can focus on the actual analytics.
<br><br>
Also, in the interest of time as well as computational resource efficiency, we have limited the variables provided to you and the number of records you have in your file.  This will make the data easier for you to manage as apposed to the 1000+ variables and 1000000+ records that will be available to you in a production environment.  As mentioned before we are more looking to assess your thought process not burn up your personal computer. You are provided below with the description of some client’s fields and the most other fields are third-party data that we received without descriptions.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
train_df = pd.read_csv('train.csv', delimiter='|')
test_df = pd.read_csv('test.csv', delimiter='|')

In [10]:
train_df.head()

,Unnamed: 0,VC_ID,VC_DATEMAIL,VC_DATERESP,VC_GIFT,VC_CATHA,VC_CATHMC,PX_WO_CRR,PX_WO_CRR6MTH,ax_donation_contribution_c,...,ax_scuba_diving,ax_adult_age_ranges_present_in_household_unknown_gender_18_24,ax_childrens_age_ranges_present_in_hhld_reseller_age_00_02_female,ax_home_owner_type_detail_rp,ax_home_market_value_deciles_estimated_rp,ax_home_property_type_rp,PX_WO_PMD,ax_home_loan_date_3_rp_year,PX_WO_PR,ax_sex
0,0,326417533784,2012-08-11,NaN,NaN,8,8,0.2754,0.2754,1.0,...,NaN,NaN,NaN,O,10.0,R,1998-07-01,NaN,1998-07-22,0.0
1,1,472446525086,2017-08-01,NaN,NaN,3,3,0.0526,0.0526,NaN,...,NaN,NaN,NaN,O,4.0,R,2011-10-03,NaN,2013-08-15,0.0
2,2,438087357784,2017-01-10,NaN,NaN,8,9,0.1111,0.1111,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2011-03-28,NaN,2011-05-06,1.0
3,3,472446537663,2017-08-01,NaN,NaN,7,8,0.1058,0.1058,1.0,...,NaN,NaN,NaN,O,7.0,R,2000-04-10,NaN,2000-09-21,0.0
4,4,326417580816,2012-08-11,NaN,NaN,3,5,0.1117,0.1117,1.0,...,NaN,NaN,NaN,O,1.0,R,2003-08-18,NaN,2003-09-11,NaN
